<a href="https://colab.research.google.com/github/ArtemKrayevskiy/Weather_AI/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/myshell-ai/MeloTTS.git
%cd MeloTTS
!pip install -e .
!python -m unidic download

%cd /content

!pip install --upgrade pip
!pip install virtualenv
!pip install --upgrade transformers datasets[audio] accelerate
!pip install -q torch accelerate bitsandbytes sentence-transformers faiss-gpu langchain langchain-community
!python -m pip install pypdf


Cloning into 'MeloTTS'...
remote: Enumerating objects: 429, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 429 (delta 205), reused 157 (delta 157), pack-reused 163 (from 1)
Receiving objects: 100% (429/429), 6.05 MiB | 4.00 MiB/s, done.
Resolving deltas: 100% (222/222), done.
/content/MeloTTS
Obtaining file:///content/MeloTTS
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 48.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install gradio

In [3]:
import gradio as gr
import requests
from datetime import datetime, timedelta
import spacy
import re
from geopy.geocoders import Nominatim
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

In [4]:
import os

from urllib.request import urlretrieve

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [5]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnablePassthrough

In [6]:
model_name = 'HuggingFaceH4/zephyr-7b-beta'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

modelLM = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [7]:
import spacy
from datetime import datetime, timedelta
import re

nlp = spacy.load("en_core_web_sm")


ordinal_mapping = {
    "first": 1, "second": 2, "third": 3, "fourth": 4,
    "fifth": 5, "sixth": 6, "seventh": 7, "eighth": 8,
    "ninth": 9, "tenth": 10, "eleventh": 11, "twelfth": 12,
    "thirteenth": 13, "fourteenth": 14, "fifteenth": 15,
    "sixteenth": 16, "seventeenth": 17, "eighteenth": 18,
    "nineteenth": 19, "twentieth": 20, "twenty first": 21,
    "twenty second": 22, "twenty third": 23, "twenty fourth": 24,
    "twenty fifth": 25, "twenty sixth": 26, "twenty seventh": 27,
    "twenty eighth": 28, "twenty ninth": 29, "thirtieth": 30,
    "thirty first": 31
}

def extract_city_date_time(user_input):
    doc = nlp(user_input)
    city = None
    date = None
    time = None

    time_of_day = {
        "morning": "08:00",
        "afternoon": "14:00",
        "evening": "19:00",
        "night": "22:00"
    }

    for ent in doc.ents:
        if ent.label_ == "GPE":
            city = ent.text
        elif ent.label_ == "DATE":
            date_text = ent.text.lower()
            today = datetime.now()

            if "today" in date_text:
                date = today.strftime("%Y-%m-%d")
            if "tomorrow" in date_text:
                date = (today + timedelta(days=1)).strftime("%Y-%m-%d")
            if "yesterday" in date_text:
                date = (today - timedelta(days=1)).strftime("%Y-%m-%d")
            if re.search(r'(next|this)?\s*(\w+)', date_text):

                day_match = re.search(r'(next|this)?\s*(\w+)', date_text)
                if day_match:
                    modifier, day_name = day_match.groups()
                    day_name = day_name.capitalize()
                    days_of_week = {
                        "Monday": 0, "Tuesday": 1, "Wednesday": 2,
                        "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6
                    }

                    if day_name in days_of_week:
                        today_day_number = today.weekday()
                        target_day_number = days_of_week[day_name]

                        if modifier == "next":
                            days_until_target = (target_day_number - today_day_number + 7)
                            if days_until_target == 0:
                                days_until_target = 7
                        else:
                            days_until_target = (target_day_number - today_day_number + 7) % 7
                            if days_until_target == 0:
                                days_until_target = 7

                        target_date = today + timedelta(days=days_until_target)
                        date = target_date.strftime("%Y-%m-%d")
            if re.search(r'(\b(?:first|second|third|fourth|fifth|sixth|seventh|eighth|ninth|tenth|eleventh|twelfth|thirteenth|fourteenth|fifteenth|sixteenth|seventeenth|eighteenth|nineteenth|twentieth|twenty first|twenty second|twenty third|twenty fourth|twenty fifth|twenty sixth|twenty seventh|twenty eighth|twenty ninth|thirtieth|thirty first)\b)\s+of\s+(\w+)', user_input):
                match = re.search(r'(\b(?:first|second|third|fourth|fifth|sixth|seventh|eighth|ninth|tenth|eleventh|twelfth|thirteenth|fourteenth|fifteenth|sixteenth|seventeenth|eighteenth|nineteenth|twentieth|twenty first|twenty second|twenty third|twenty fourth|twenty fifth|twenty sixth|twenty seventh|twenty eighth|twenty ninth|thirtieth|thirty first)\b)\s+of\s+(\w+)', user_input)
                if match:

                    day_text = match.group(1)
                    month_text = match.group(2)

                    day = ordinal_mapping.get(day_text)

                    month = datetime.strptime(month_text.capitalize(), "%B").month

                    year = datetime.now().year

                    if day:
                        date = datetime(year, month, day).date().strftime("%Y-%m-%d")

            if date is None:
                date = ent.text

    for period, default_time in time_of_day.items():
        if period in user_input.lower():
            time = default_time
            break

    if city and date:
        return {"city": city, "date": date, "time": time}
    else:
        return {"error": "City or date not found. Please specify the city and date."}

user_input = "What was the weather in Los Angeles on twenty third of September?"
result = extract_city_date_time(user_input)
print(result)

{'city': 'Los Angeles', 'date': '2024-09-23', 'time': None}


In [ ]:
from google.colab import userdata


import requests
from datetime import datetime
from geopy.geocoders import Nominatim



def get_visual_crossing_weather(api_key, location_input, date, time):


    geolocator = Nominatim(user_agent="city_locator")
    location = geolocator.geocode(location_input)
    latitude = location.latitude
    longitude = location.longitude
    if time:
      time += ":00"


    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{latitude},{longitude}/{date}?key={api_key}&unitGroup=metric"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if "days" in data:
          if time == None:
            this_day = data["days"][0]
            return {'city' : location_input,
                    'date' : date,
                    'tempmax': this_day['tempmax'],
                    'tempmin': this_day['tempmin'],
                    'humidity' : this_day['humidity'],
                    'temp' : this_day['temp'],
                    'conditions' : this_day['conditions'],
                    'description' : this_day['description'],
                    'visibility' : this_day['visibility']}
          else:
            for i in data["days"][0]["hours"]:
              if i["datetime"] == time:
                this_time = i
                return {'city' : location_input,
                        'date' : date,
                        'time_of_day' : time,
                        'humidity' : this_time['humidity'],
                        'temp' : this_time['temp'],
                        'feelslike' : this_time['feelslike'],
                        'conditions' : this_time['conditions'],
                        'visibility': this_time['visibility']}



        return {"error" : "You forgot to specify city or date"}
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")


weather_api_key = userdata.get('WEATHER_API')
this_func_result = get_visual_crossing_weather(weather_api_key, result["city"], result["date"], result["time"])
this_func_result
# structured_query = extract_city_and_date(question)
# data = get_visual_crossing_weather(weather_api_key, structured_query.get("city"), structured_query.get("date"))
# data

{'city': 'Los Angeles',
 'date': '2024-09-23',
 'tempmax': 23.0,
 'tempmin': 16.2,
 'humidity': 85.0,
 'temp': 18.7,
 'conditions': 'Partially cloudy',
 'description': 'Partly cloudy throughout the day.',
 'visibility': 9.2}

In [9]:
text_generation_pipeline = pipeline(
    model=modelLM,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>

Communicate with user in a friendly manner:
{context}

</s>
<|user|>
{input}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "input"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

Device set to use cuda:0
<ipython-input-9-0dd42f258e4c>:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [10]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"
modelSpeech2Text = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)

modelSpeech2Text.to(device)
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [11]:
%cd MeloTTS

/content/MeloTTS


In [12]:
from melo.api import TTS

modelText2Speech = TTS(language='EN', device=device)
speaker_ids = modelText2Speech.hps.data.spk2id

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


checkpoint.pth:   0%|          | 0.00/208M [00:00<?, ?B/s]

/content/MeloTTS/melo/download_utils.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(ckpt_path, map_location=device)


In [13]:
%cd /content

/content


In [14]:
!pip install TTS

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 71.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━

In [25]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [17]:
from TTS.api import TTS

tts = TTS("tts_models/multilingual/multi-dataset/your_tts", gpu=True)

# response_text = "I think, i need some other text for example texting, what is the weather like today, or something like that."

# tts.tts_to_file(
#     text=response_text,
#     speaker_wav="vitalik_voice_sample.wav",
#     language="en",
#     file_path="output.wav"
# )

/usr/local/lib/python3.10/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


/usr/local/lib/python3.10/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:512
 | > power:1.5
 | > preemphasis:0.97
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:False
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:False
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:True
 | > db_level:-27.0
 | > stats_path:None
 | > base:10
 | > hop_length:160
 | > win_length:400
 > External Speaker Encoder Loaded !!
 > initialization of language-embedding layers.
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10

In [26]:
import random

def generate_speech_to_text(audio):

    if not os.path.exists(audio):
        raise FileNotFoundError(f"Audio file not found: {audio}")

    pipe = pipeline(
      "automatic-speech-recognition",
      model=modelSpeech2Text,
      tokenizer=processor.tokenizer,
      feature_extractor=processor.feature_extractor,
      torch_dtype=torch_dtype,
      device=device,
      )

    result = pipe(audio)
    return  result["text"]


def generate_text_to_speech(text):
    output_path = 'response_audio.wav'
    speed = 1.0
    speaker_ids = modelText2Speech.hps.data.spk2id
    modelText2Speech.tts_to_file(text, speaker_ids['EN-US'], output_path, speed=speed)
    return output_path

def generate_joke_response(joke):
    output_path = 'response_audio.wav'
    tts.tts_to_file(
    text=joke,
    speaker_wav="vitalik_voice_sample2.wav",
    language="en",
    file_path=output_path,
    speed=0.8
    )
    return output_path



def generate_weather_response(weather_data):
    result = extract_city_date_time(weather_data)
    if "error" in result.keys():
      prompt = f"Generate a natural language response for such error: {result['error']}"
    else:
      weather_data = get_visual_crossing_weather(weather_api_key, result["city"], result["date"], result["time"])
      if result["time"] == None:
        prompt = f"Here is the weather information for {weather_data['city']} on {weather_data['date']}:\n" \
                f"Temperature: {weather_data['temp']}°C, Max: {weather_data['tempmax']}°C, Min: {weather_data['tempmin']}°C\n" \
                f"Humidity: {weather_data['humidity']}%\nConditions: {weather_data['conditions']}\n" \
                f"Description: {weather_data['description']}\nVisibility: {weather_data['visibility']} km.\n\n" \
                f"Generate a natural language response summarizing the weather and specifying this date: {weather_data['date']} Also provide \
              reccommendation on clothes which are suitable for this weather. Answer:"
      else:
          prompt = f"Here is the weather information for {weather_data['city']} on {weather_data['date']} at {weather_data['time_of_day']}:\n" \
              f"Temperature: {weather_data['temp']}°C, feels like: {weather_data['feelslike']}\n" \
              f"Humidity: {weather_data['humidity']}%\nConditions: {weather_data['conditions']}\n" \
              f"Visibility: {weather_data['visibility']} km.\n\n" \
              f"Generate a natural language response summarizing the weather and specifying this date: {weather_data['date']} and this time{weather_data['time_of_day']}. Also provide \
              reccommendation on clothes which are suitable for this weather. Answer:"

    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    device = modelLM.device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():
        outputs = modelLM.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=300,
            num_return_sequences=1,
            do_sample=True,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


def chat_with_speech(audio_path):
    print(audio_path)
    user_input = generate_speech_to_text(audio_path)
    user_input_lower = user_input.lower()
    print(f"User input (from speech): {user_input}")

    if "weather" in user_input_lower:
        response_text = generate_weather_response(user_input).split("Answer:")[-1]

    elif "joke" in user_input_lower:
       jokes = ["I told my wife she should embrace her mistakes.She gave me a hug." , \
             "Today, I asked my phone 'Siri, why am I still single?' and it activated the front camera.",\
             "Why did the magician adopt a Labrador? Because every time he said 'Labracadabra', it fetched his wand!",\
             "Why did the hairdresser win the race?He knew a shortcut",\
             "what is the name of a crow that has landed on an electric wire? Electrocar",\
             "How are marriages like algebra? When you look at your x, you can’t help but wonder y.",\
             "My boss said, “Have a good day!” So, I went home.",\
             "Why was six afraid of seven? Because seven ate nine",\
             "Two cannibals drank, but only one ate."
             ]

       response_text = random.choice(jokes)
       response_audio = generate_joke_response(response_text)

       return response_text, response_audio
    else:
        response_text = llm_chain.invoke({"context":"", "input": user_input}).split("Answer:")[-1]

    response_audio = generate_text_to_speech(response_text)

    return response_text, response_audio


with gr.Blocks() as demo:
    gr.Markdown("# Voice-Enabled Weather Assistant")

    audio_input = gr.Audio(sources="microphone",type="filepath", label="Speak your question")

    text_output = gr.Textbox(label="Chatbot Response")
    audio_output = gr.Audio(label="Audio Response")

    submit_btn = gr.Button("Submit")

    submit_btn.click(
        fn=chat_with_speech,
        inputs=audio_input,
        outputs=[text_output, audio_output]
    )

demo.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://44936f8c0b0e55b9b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Device set to use cuda:0


/tmp/gradio/978e06603bb8a3813892ce4bc3c54b79f4e9745a1cee1f823293b2e6e282440f/audio.wav


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


User input (from speech):  What is the weather like in Lviv today?
 > Text split to sentences.
It's going to be a chilly day in Lviv on December 18th, 2024. The temperature will range from 4. 4 to 8. 1 degrees Celsius, with early morning rain that will clear up to leave a partly cloudy sky. It's best to dress in layers, as you'll want a coat or jacket to keep warm,
but you may not need it once the sun comes out. Be sure to bring an umbrella just in case the rain persists, and consider packing a hat and gloves for added warmth. Overall, a mix of long-sleeved shirts, pants, and closed-toe shoes should do the trick for staying comfortable throughout the day.
 > ===========================



  0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://44936f8c0b0e55b9b2.gradio.live
